# SBERT

#### 1.Library

In [2]:
import pandas as pd
import numpy as np

In [ ]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

In [21]:
!os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
!os.environ["CUDA_VISIBLE_DEVICES"]= "#"

/bin/bash: os.environ[CUDA_DEVICE_ORDER]=PCI_BUS_ID: command not found
/bin/bash: os.environ[CUDA_VISIBLE_DEVICES]=: command not found


In [22]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())

Device: cuda
Current cuda device: 0


In [1]:
!nvidia-smi

Sun Jun 12 11:51:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   37C    P0    62W / 300W |   1267MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [2]:
!kill -9 55181

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#### 2.Data Load

- train dataset

In [4]:
code_folder = '/home/ubuntu/HwangJaewon/DL_class/code/'
problem_folders = os.listdir(code_folder)

train = pd.read_csv("/home/ubuntu/HwangJaewon/DL_class/sample_train.csv")
train.head()
len(train)

17970

- Test dataset for submission 

In [5]:
test = pd.read_csv("/home/ubuntu/HwangJaewon/DL_class/test.csv")
test.head()
len(test)

179700

- Validation dataset

In [6]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, test_size = 0.2, random_state = 42, stratify = train['similar'])

In [7]:
valid.head()

,code1,code2,similar
2885,"N=int(input())\nA=list(map(int,input().split()...","n = int(input())\na = list(map(int,input().spl...",1
14761,K = int(input())\nnumbers = set([])\nq = set(l...,from collections import deque\n\nK = int(input...,1
6936,"d = int(input())\nc = list(map(int,input().spl...","import numpy as np\n\nA=[list(map(int, input()...",0
15368,x = int(input())\n\ndef score(x):\n if 400 <=...,S = input()\nL = S.split()\n\na = L[0]\nb = L[...,0
15218,import numpy as np\nimport sys\nread = sys.std...,import math\n \nN = int(input())\n \nM = int(m...,0


In [8]:
len(valid)

3594

#### 3.Modeling

In [9]:
# module import
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [10]:
# model
model = CrossEncoder('cross-encoder/stsb-roberta-base', num_labels=1)

In [11]:
# data를 InputExample로 묶어준 후, torch dataloader로 묶기
train_examples = []
valid_examples = []
for idx, row in tqdm(train.iterrows()):
    train_examples.append(InputExample(texts = [row['code1'], row['code2']], label = row['similar']))

for idx, row in tqdm(valid.iterrows()):
    valid_examples.append(InputExample(texts = [row['code1'], row['code2']], label = row['similar']))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
# Data Loader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
valid_dataloader = CEBinaryClassificationEvaluator.from_input_examples(valid_examples)
train_dataloader

In [13]:
# Model fit
model.fit(train_dataloader=train_dataloader,
          evaluator=valid_dataloader,
          epochs=1,
          warmup_steps=100)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/899 [00:00<?, ?it/s]

In [14]:
# Score
model.best_score

0.9844679284869023

In [15]:
# model save
model.save('dacon_2022/')

#### 4.Predict

In [16]:
test_examples = []
for idx, row in tqdm(test.iterrows()):
    test_examples.append([row['code1'], row['code2']])

0it [00:00, ?it/s]

In [17]:
LEN_TEST = len(test_examples)
LEN_TEST

179700

In [18]:
# test
preds = []
for idx in tqdm(range(LEN_TEST)):
    preds.append(model.predict(test_examples[idx]))

  0%|          | 0/179700 [00:00<?, ?it/s]

#### 5.Submission

In [20]:
submission = pd.read_csv("/home/ubuntu/HwangJaewon/DL_class/sample_submission.csv")

In [21]:
submission.similar = preds

In [22]:
submission.similar = submission.similar.apply(lambda x : 0 if x < 0.5 else 1)

In [23]:
t = pd.Timestamp.now()
fname = f"submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
submission.to_csv(f'{fname}', index=False)
print(f"'{fname}' is ready to submit.")

'submission_06111122.csv' is ready to submit.
